In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datasets
import transformers
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import hashlib

In [3]:
ds_full = datasets.load_dataset("franlucc/py_typeinf_rename_vars_unfiltered", split="train")
ds_neg = datasets.load_dataset("franlucc/py_typeinf_rename_vars", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/283373 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8319 [00:00<?, ? examples/s]

In [4]:
def get_hex(x):
    return hashlib.sha256(bytes(x, "utf-8")).hexdigest()

ds_full = ds_full.map(lambda x: {**x, "myid": get_hex(x["mutated_program"])})
ds_neg = ds_neg.map(lambda x: {**x, "myid": get_hex(x["mutated_program"])})
ignore = set(ds_neg["myid"])
ds_pos = ds_full.filter(lambda x: not x["myid"] in ignore)

Map:   0%|          | 0/283373 [00:00<?, ? examples/s]

Map:   0%|          | 0/8319 [00:00<?, ? examples/s]

Filter:   0%|          | 0/283373 [00:00<?, ? examples/s]

In [5]:
ds_pos, ds_neg, ds_full

(Dataset({
     features: ['mutated_program', 'mutations', 'key', 'prefix', 'suffix', 'middle', 'correct', 'model', 'fim_type', 'fim_program', 'hexsha', 'myid'],
     num_rows: 275054
 }),
 Dataset({
     features: ['mutated_program', 'mutations', 'key', 'prefix', 'suffix', 'middle', 'correct', 'model', 'fim_type', 'fim_program', 'hexsha', 'mutated_generated_text', 'myid'],
     num_rows: 8319
 }),
 Dataset({
     features: ['mutated_program', 'mutations', 'key', 'prefix', 'suffix', 'middle', 'correct', 'model', 'fim_type', 'fim_program', 'hexsha', 'myid'],
     num_rows: 283373
 }))

In [6]:
modelname = "/home/arjun/models/starcoderbase-1b"

In [7]:
from codetrace.interp_utils import *
model = LanguageModel(modelname, device_map="cuda:0")

In [8]:
from codetrace.utils import placeholder_to_std_fmt, STARCODER_FIM
from tqdm import tqdm
import os

# collect activations
max_size = 1000
prompts_pos = [placeholder_to_std_fmt(x["mutated_program"], STARCODER_FIM) for x in ds_pos.shuffle(42).select(range(max_size))]
prompts_neg = [placeholder_to_std_fmt(x["mutated_program"], STARCODER_FIM) for x in ds_neg.shuffle(42).select(range(max_size))]

batchsize = 5

def save_activations(data, batchsize, outdir):
    os.makedirs(outdir, exist_ok=True)
    batched = [data[i:i+batchsize] for i in range(0, len(data), batchsize)]
    for i,batch in tqdm(enumerate(batched), desc="Collecting activations...", total=len(batched)):

        hs = collect_hidden_states_at_tokens(model, batch, "<fim_middle>")

        torch.save(hs, f"{outdir}/activations_{i}.pt")

save_activations(prompts_pos, 5, "activations_pos_vars")
save_activations(prompts_neg, 5, "activations_neg_vars")

In [9]:
del(model)

In [10]:
from glob import glob
def load_data(activdir):
    tensor = None
    for f in glob(f"{activdir}/*.pt"):
        a = torch.load(f)
        if tensor == None:
            print(a.shape)
            tensor = a
        else:
            tensor = torch.cat((tensor, a), dim=1)
    return tensor

data_pos = load_data("activations_pos_vars")
data_neg = load_data("activations_neg_vars")

torch.Size([24, 5, 1, 2048])
torch.Size([24, 5, 1, 2048])


In [41]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
def run_cls(classifier, layers, max_size=1000, do_fit=True):
    print(data_pos.shape, data_neg.shape)
    labels = [1]*(len(layers)*max_size) + [0]*(len(layers)*max_size)
    data = []
    for split in [data_pos, data_neg]:
        for prompt in range(max_size):
            for layer in layers:
                data.append(split[layer, prompt, -1, :].flatten().numpy())
    print(len(data), data[0].shape, len(labels))
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
    print(len(X_train), len(X_test))
    if do_fit:
        classifier.fit(X_train, y_train)
    else:
        classifier.fit([np.random.rand(2048),np.random.rand(2048)],[0,1])
    y_pred = classifier.predict(X_test)
    y_train_pred = classifier.predict(X_train)
    print("test acc:", accuracy_score(y_test, y_pred))
    print("train acc:", accuracy_score(y_train, y_train_pred))
    print("test prec:", precision_score(y_test, y_pred))
    print("train prec:", precision_score(y_train, y_train_pred))
    print("test recall:", recall_score(y_test, y_pred))
    print("train recall:", recall_score(y_train, y_train_pred))
    print("test f1:", f1_score(y_test, y_pred))
    print("train f1:", f1_score(y_train, y_train_pred))
    return classifier
            

In [12]:
classifier = LogisticRegression(max_iter=10000, verbose=1)
cls012 = run_cls(classifier, [0,1,2])

torch.Size([24, 1000, 1, 2048]) torch.Size([24, 1000, 1, 2048])
6000 (2048,) 6000
4800 1200
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         2049     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  5.71005D-02

At iterate   50    f=  5.60613D-01    |proj g|=  1.52471D-02


 This problem is unconstrained.



At iterate  100    f=  5.52056D-01    |proj g|=  4.53816D-03

At iterate  150    f=  5.49372D-01    |proj g|=  1.11906D-03

At iterate  200    f=  5.48890D-01    |proj g|=  8.10202D-04

At iterate  250    f=  5.48766D-01    |proj g|=  3.42584D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 2049    277    318      1     0     0   7.532D-05   5.488D-01
  F =  0.54875083022426807     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
test acc: 0.7025
train acc: 0.768125


In [43]:
classifier = LogisticRegression(max_iter=1, verbose=0, random_state=0)
cls14 = run_cls(classifier, [14], do_fit=False)

torch.Size([24, 1000, 1, 2048]) torch.Size([24, 1000, 1, 2048])
2000 (2048,) 2000
1600 400
test acc: 0.5875
train acc: 0.573125
test prec: 0.7297297297297297
train prec: 0.6915584415584416
test recall: 0.271356783919598
train recall: 0.26591760299625467
test f1: 0.3956043956043956
train f1: 0.3841298467087466


/home/franlucc/venvs/gpu/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
classifier = LogisticRegression(max_iter=10000, verbose=1)
cls14 = run_cls(classifier, list(range(24)))

torch.Size([24, 1000, 1, 2048]) torch.Size([24, 1000, 1, 2048])
48000 (2048,) 48000
38400 9600
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         2049     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  2.38989D-01


 This problem is unconstrained.



At iterate   50    f=  2.94189D-01    |proj g|=  1.96134D-02

At iterate  100    f=  2.47426D-01    |proj g|=  1.78599D-02

At iterate  150    f=  2.31618D-01    |proj g|=  7.70364D-03

At iterate  200    f=  2.25892D-01    |proj g|=  1.76276D-02

At iterate  250    f=  2.22718D-01    |proj g|=  1.68225D-03

At iterate  300    f=  2.21159D-01    |proj g|=  2.81239D-03

At iterate  350    f=  2.20350D-01    |proj g|=  1.80445D-03

At iterate  400    f=  2.19905D-01    |proj g|=  2.30721D-03

At iterate  450    f=  2.19723D-01    |proj g|=  1.44114D-03

At iterate  500    f=  2.19629D-01    |proj g|=  5.36302D-04

At iterate  550    f=  2.19582D-01    |proj g|=  5.30290D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F   

In [15]:
classifier = LogisticRegression(max_iter=10000, verbose=1)
cls18 = run_cls(classifier, [18])

torch.Size([24, 1000, 1, 2048]) torch.Size([24, 1000, 1, 2048])
2000 (2048,) 2000
1600 400
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         2049     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  6.45860D-01

At iterate   50    f=  2.58448D-02    |proj g|=  2.33559D-03

At iterate  100    f=  2.20436D-02    |proj g|=  1.81976D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 2049    134    157      1     0     0   9.044D-05   2.184D-02
  F =   2.1837785572766936E-002

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_

 This problem is unconstrained.


## rename types py

0,1,2
test acc: 0.5775
train acc: 0.678125

14/18?
test acc: 0.8575
train acc: 0.999375

## Other viz stuff

In [60]:
print(activations.shape)

torch.Size([24, 1, 22, 2048])


In [61]:
outs = []
for i in range(24):    
    out = tuned_lens_viz(prompt, model, original_decoder, layer=i, top_k=10, activations=activations)
    outs.append(out)

[
 {
  "<fim_suffix>": 0.009965,
  "UGH": 0.005806,
  "Referencia": 0.003597,
  "ingleton": 0.003126,
  "Desde": 0.003016,
  "utoff": 0.002464,
  "Punto": 0.002128,
  "Tipos": 0.002108,
  "ialect": 0.001949,
  "Campos": 0.001922,
  "token": "<fim_prefix>"
 },
 {
  "ynt": 0.112886,
  "define": 0.041574,
  "oolean": 0.025977,
  "quares": 0.022676,
  "def": 0.019737,
  "ymb": 0.016038,
  "dcd": 0.015519,
  " padr": 0.014287,
  "geb": 0.010721,
  "angerous": 0.009317,
  "token": "def"
 },
 {
  " is": 0.132763,
  "veloper": 0.037602,
  "nt": 0.030998,
  "aeb": 0.017512,
  " not": 0.01189,
  "tsch": 0.010046,
  "eec": 0.008536,
  "aec": 0.00736,
  "bbf": 0.005836,
  "lfw": 0.005611,
  "token": " is"
 },
 {
  "ingleton": 0.0447,
  "creen": 0.026022,
  "neer": 0.016285,
  "cerning": 0.014213,
  "awl": 0.011756,
  "oggle": 0.009008,
  "ixo": 0.008837,
  "cheme": 0.008291,
  "keh": 0.007245,
  "kerja": 0.006176,
  "token": "_"
 },
 {
  "pal": 0.399395,
  "Hq": 0.042371,
  "Kj": 0.026322,
  "Uq":

In [62]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(modelname)
import os
os.makedirs("lens_view_3_patch", exist_ok=True)
toks = tokenizer.tokenize(prompt)
wrap_htmls(outs, "lens_view_3_patch", labels = [f"Layer {i}" for i in range(24)])

dict_keys(['template.html', 'scripts.js', 'styles.css'])


'<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\n<title>Tabbed Content</title>\n<!-- add stylesheet -->\n<link rel="stylesheet" href="styles.css">\n</head>\n<body>\n\n<div class="tabs">\n<button class="tablinks" onclick="openTab(event, \'tab0\')">Layer Layer 0</button>\n\n<button class="tablinks" onclick="openTab(event, \'tab1\')">Layer Layer 1</button>\n\n<button class="tablinks" onclick="openTab(event, \'tab2\')">Layer Layer 2</button>\n\n<button class="tablinks" onclick="openTab(event, \'tab3\')">Layer Layer 3</button>\n\n<button class="tablinks" onclick="openTab(event, \'tab4\')">Layer Layer 4</button>\n\n<button class="tablinks" onclick="openTab(event, \'tab5\')">Layer Layer 5</button>\n\n<button class="tablinks" onclick="openTab(event, \'tab6\')">Layer Layer 6</button>\n\n<button class="tablinks" onclick="openTab(event, \'tab7\')">Layer Layer 7</button>\n\n<button class="tablinks" onclick="